In [1]:
import sys 
sys.path.append('../../../')
sys.path.append('../../')
sys.path.append('../')

from core.active_learning import *
from core.auto_labeling import *
from utils.logging_utils import * 
from datasets import dataset_factory 
from datasets.dataset_utils import * 
from utils.counting_utils import *  
from utils.common_utils import * 
from utils.vis_utils import *
from core.run_lib import * 
import copy 
import random 

root_dir = '../../'

config_file = f'{root_dir}/configs/neurips23/circles/active_learning_circles.yaml'
conf        = OmegaConf.load(config_file)
logger      = get_logger(f'{root_dir}/temp/logs/al.log','PL',level=logging.DEBUG)
#logger = get_logger('../../temp/logs/pl.log','PL')

train_conf = conf['training_conf']


print(train_conf)

{'train_set_frac': 0.1, 'loss_tolerance': 1e-05, 'max_epochs': 1000, 'normalize_weights': False, 'C': 1, 'stopping_criterion': 'val_err_threshold', 'val_err_threshold': 0.3, 'ckpt_load_path': 'None', 'ckpt_save_path': 'None', 'train_from_scratch': True, 'train_from_ckpt': False, 'ckpt_dir': './', 'probability': True}


In [2]:
set_seed(7)

In [3]:
dm = DataManager(conf,logger)
len(dm.ds_std_train), len(dm.ds_std_val)

act_lrn = ActiveLearning(conf,dm,logger)

act_lrn.init()

lst_epoch_out = act_lrn.run_al_loop()

[11/25/2023 03:33:48 PM : INFO  : data_manag : ] : Loaded dataset synth_concenteric_circles
[11/25/2023 03:33:48 PM : INFO  : data_manag : ] : Std train size: 8000 and Std. Val. Size:2000
[11/25/2023 03:33:48 PM : DEBUG : active_lea : ] : Stop Criterion max_num_train_pts
[11/25/2023 03:33:48 PM : DEBUG : active_lea : ] : Unlabeled Count In check_stop_criterion 8000
[11/25/2023 03:33:48 PM : DEBUG : active_lea : ] : cur_query_count= 0 and max_query_count=500
[11/25/2023 03:33:48 PM : DEBUG : active_lea : ] : err_th 1.0 and val_err_threshold 0.01
[11/25/2023 03:33:48 PM : DEBUG : active_lea : ] : err_th 1.0 and val_err_threshold 0.01
[11/25/2023 03:33:48 PM : INFO  : active_lea : ] : ===========================================================================
[11/25/2023 03:33:48 PM : DEBUG : active_lea : ] : ========================= BEGIN EPOCH 0 ============================
[11/25/2023 03:33:48 PM : DEBUG : active_lea : ] : Number of unalabeled points  :8000
[11/25/2023 03:33:48 PM : I

In [4]:

auto_lbl_conf = conf['auto_lbl_conf']

auto_lbl_conf['method_name']= 'all' 

meta_df_cp = copy.deepcopy(dm.meta_df) 

auto_labeler = AutoLabeling(conf,dm,act_lrn.cur_clf,logger)
out = auto_labeler.run()
counts_all = dm.get_auto_labeling_counts()

auto_lbl_conf['method_name']= 'selective' 

dm.meta_df = meta_df_cp 

auto_labeler = AutoLabeling(conf,dm,act_lrn.cur_clf,logger)
out = auto_labeler.run()
counts_sel = dm.get_auto_labeling_counts()


[11/25/2023 03:33:50 PM : INFO  : auto_label : ] : ========================= Begin Auto-Labeling all ==========================
[11/25/2023 03:33:50 PM : DEBUG : auto_label : ] : Auto Labeling Conf : {'method_name': 'all', 'score_type': 'confidence', 'class_wise': 'independent', 'auto_label_err_threshold': 0.01, 'C_1': 0.25, 'ucb': 'sigma', 'fast': True}
[11/25/2023 03:33:50 PM : INFO  : auto_label : ] : Number of unlabeled points : 7748
[11/25/2023 03:33:50 PM : INFO  : auto_label : ] : Num auto labeled points : 7748 
[11/25/2023 03:33:50 PM : INFO  : auto_label : ] : Num validation pts to remove : 2000
[11/25/2023 03:33:50 PM : INFO  : auto_label : ] : ============================== Done Auto-Labeling ==============================
[11/25/2023 03:33:50 PM : INFO  : auto_label : ] : ========================= Begin Auto-Labeling selective ==========================
[11/25/2023 03:33:50 PM : DEBUG : auto_label : ] : Auto Labeling Conf : {'method_name': 'selective', 'score_type': 'confid

In [5]:
print(counts_all)
print(counts_sel)

{'auto_labeled_acc': 0.48877129581827566, 'coverage_1': 0.9685, 'coverage_2': 0}
{'auto_labeled_acc': 0.9981464318813716, 'coverage_1': 0.134875, 'coverage_2': 0}


In [6]:
act_lrn_out = {'lst_epoch_out':lst_epoch_out, 'dm':dm}

import pickle 
out_dir = f'{root_dir}/outputs/circles/vis/'
try:
    os.makedirs(out_dir)    
except OSError:
    pass

fpath_out = f'{out_dir}/act_lrn_vis_out.pkl'

with open(fpath_out, 'wb') as handle:
    pickle.dump(act_lrn_out, handle, protocol=pickle.HIGHEST_PROTOCOL) 